# Using Open-source HuggingFace Models (Based on Gemma-7b-it) to Generate QAs from Raw Data in JSON format

In this example, we will show you how to generate question-answers (QAs) from given text strings using open-source Huggingface models **using [Gemma-7b-it](https://huggingface.co/google/gemma-7b-it)** via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

Next, you will need a valid Huggingface Token to access the Gemma model. Once you have the token, set it as the environment variable `HF_token` within a `.env` file in the root directory of this repository.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy tiktoken

### Import dependency

In [12]:
from IPython.display import Markdown, display

from dotenv import load_dotenv
from uniflow.flow.flow_factory import FlowFactory
from uniflow.flow.client import TransformClient
from uniflow.flow.config import HuggingfaceModelConfig, TransformQAHuggingFaceJsonFormatConfig
from uniflow.flow.config import GemmaTransformConfig
from uniflow.op.prompt import Context
from uniflow.op.prompt import PromptTemplate
from pprint import pprint
import time

load_dotenv()

True

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class. Let's create raw text for context.

In [4]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """Snoopy can be selfish, gluttonous, and lazy at times, and occasionally mocks his owner, Charlie Brown. 
But on the whole, he shows great love, care, and loyalty for his owner (even though he cannot even remember 
his name and always refers to him as "the round-headed kid"). In the 1990s comic strips, he is obsessed 
with cookies, particularly the chocolate-chip variety. This, and other instances in which he indulges in large 
chocolate-based meals and snacks, indicate that chocolate is not poisonous to Snoopy, the way it is for real dogs.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\)."""
]

raw_context_input = raw_context_input

Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [5]:

input_data = [
    Context(context=data)
    for data in raw_context_input
]

print("sample size of processed input data: ", len(input_data))

print("\nExample uniflow context data:")
pprint(input_data[:3])


sample size of processed input data:  3

Example uniflow context data:
[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='Snoopy can be selfish, gluttonous, and lazy at times, and occasionally mocks his owner, Charlie Brown. \nBut on the whole, he shows great love, care, and loyalty for his owner (even though he cannot even remember \nhis 

### Use LLM to generate data

In this example, we will use the [GemmaTransformConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/config.py)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here in this example, we use define our own `PromptTemplate` to the `GemmaTransformConfig`, but you can use your default instructions and examples instead if you want.


In [6]:
guided_prompt = PromptTemplate(
        instruction="Generate one question and its corresponding answer based on context. Following the format and structure of the examples below to include the same context, question, and answer in the response.",
        few_shot_prompt=[
            Context(
                context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
                question="Who published A Mathematical Theory of Communication in 1948?",
                answer="Claude E. Shannon.",
            )
        ]
)

In [7]:
config = GemmaTransformConfig(
    prompt_template=guided_prompt,
)

client = TransformClient(config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.26s/it]


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [8]:
outputs = client.run(input_data)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['<bos><start_of_turn>user\ninstruction: Generate one question and its corresponding answer based on context. Following the format and structure of the examples below to include the same context, question, and answer in the response.\ncontext: In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.\nquestion: Who published A Mathematical Theory of Communication in 1948?\nanswer: Claude E. Shannon.\ncurrent context: We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in 

100%|██████████| 1/1 [00:25<00:00, 25.44s/it]


### Process the output

Let's take a look of the generated output, which is already a list of JSON

In [16]:
print("Sample output:")

pprint(outputs[0]['output'][0]['response'][1])

Sample output:
('instruction: Generate one question and its corresponding answer based on '
 'context. Following the format and structure of the examples below to include '
 'the same context, question, and answer in the response.\n'
 'context: In 1948, Claude E. Shannon published A Mathematical Theory of\n'
 'Communication (Shannon, 1948) establishing the theory of\n'
 'information. In his article, Shannon introduced the concept of\n'
 'information entropy for the first time. We will begin our journey here.\n'
 'question: Who published A Mathematical Theory of Communication in 1948?\n'
 'answer: Claude E. Shannon.\n'
 'current context: Snoopy can be selfish, gluttonous, and lazy at times, and '
 'occasionally mocks his owner, Charlie Brown. \n'
 'But on the whole, he shows great love, care, and loyalty for his owner (even '
 'though he cannot even remember \n'
 'his name and always refers to him as "the round-headed kid"). In the 1990s '
 'comic strips, he is obsessed \n'
 'with cooki

In [17]:
text = outputs[0]['output'][0]['response'][0]

import re

def extract_last_qa(text):
    # Regular expression to match the last current context, question, and answer
    pattern = re.compile(
        r'(?i)current context:\s*(.*?)\s*(?:\*\*?Question\*\*?:|question:)\s*(.*?)\s*(?:\*\*?Answer\*\*?:|answer:)\s*(.*?)\s*(?=\Z|\n\n)',
        re.DOTALL
    )

    # Find all matches
    matches = pattern.findall(text)

    # Return the last match, if any
    if matches:
        last_match = matches[-1]
        return {
            'current_context': last_match[0].strip(),
            'question': last_match[1].strip(),
            'answer': last_match[2].strip()
        }
    else:
        return None


result = extract_last_qa(text)
if result:
    print("Current Context:", result['current_context'])
    print("Question:", result['question'])
    print("Answer:", result['answer'])
else:
    print("No question and answer found.")


Current Context: We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.

**
Question: ** What does current text suggest about protection IP Rights related with company's assets
Answer: The provided texts suggests  that Company has implemented policies designed specifically aimed at protecting their Intellectual Property(IP )Rights which includes trademark,utility & Design Patents,,

## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>